In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..")))


In [2]:
import pandas as pd
from src.demand_forecast import calculate_moving_average
from src.data_loader import load_sales, load_purchases, load_products

In [3]:
sales = load_sales('../data/sales.csv')
forecast_7d = calculate_moving_average(sales, window=7)
print(forecast_7d)

      product_id  forecast
0     0000004494  1.142857
1     0000004524  1.142857
2     0000004525  1.428571
3     0000004526  1.285714
4     0000004532  1.714286
...          ...       ...
2300  0000015765       NaN
2301  0000015766       NaN
2302  0000015767       NaN
2303  0000015768       NaN
2304  0000015771       NaN

[2305 rows x 2 columns]


In [4]:
print(sales.head(5))

   sale_date  product_id  quantity  total_amount
0 2025-02-03  0000015463       1.0          2.88
1 2025-07-08  0000013712       1.0          0.51
2 2025-02-06  0000015124       1.0          0.50
3 2025-02-13  0000014269       2.0          3.40
4 2025-01-10  0000014102       2.0          2.90
